In [ ]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import matplotlib.pyplot as plt 

urls=["https://towardsdatascience.com/how-teachers-can-use-data-science-to-analyse-students-scores-26d0b2ec5694"]
headers_={'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

for url in urls:
  page=requests.get(url, headers=headers_)
  
  soup=bs(page.content,'html.parser')

  Contents=soup.findAll("p")

Scrapped_data=[]
for content in Contents:
  Text=content.get_text()
  Scrapped_data.append([Text])

df=pd.DataFrame(Scrapped_data,columns=["Text"])

display(df)

df.to_json("student.json")

AttributeError: ignored

In [ ]:
import string
string.punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['puntuation free text']= df['Text'].apply(lambda x:remove_punctuation(x))
#Lowering the text
df['lower']= df['puntuation free text'].apply(lambda x: x.lower())

#Tokenization
from nltk.tokenize import TweetTokenizer as tt
#applying function to the column
tokenizer = tt()      
df['tokenized_text'] = df['lower'].apply(lambda x: tokenizer.tokenize(x))

#Removing stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopword = stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopword]
    return output
df['no_stopwords']= df['tokenized_text'].apply(lambda x:remove_stopwords(x))

#Lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(no_stopwords):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in no_stopwords]
    return lemm_text
df['lematizer']=df['no_stopwords'].apply(lambda x:lemmatizer(x))
display(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Text,puntuation free text,lower,tokenized_text,no_stopwords,lematizer
0,Sign up,Sign up,sign up,"[sign, up]",[sign],[sign]
1,Sign In,Sign In,sign in,"[sign, in]",[sign],[sign]
2,Sign up,Sign up,sign up,"[sign, up]",[sign],[sign]
3,Sign In,Sign In,sign in,"[sign, in]",[sign],[sign]
4,Towards Data Science,Towards Data Science,towards data science,"[towards, data, science]","[towards, data, science]","[towards, data, science]"


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
from nltk.corpus import stopwords

#Vis
import pyLDAvis
import pyLDAvis.gensim

import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
def load_data(file):
  with open(file,"r",encoding="utf-8") as f:
    data=json.load(f)
  return(data)

def write_data(file,data):
  with open(file,"w",encoding="utf-8") as f:
    json.dump(data,f,indent=4)

In [ ]:
nltk.download("stopwords")
print(stopwords)

<WordListCorpusReader in '/root/nltk_data/corpora/stopwords'>


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data=load_data("/content/student.json")["Text"]
data=data["10"]
data=data.split(".")
data

['The first step is to get data',
 ' The teacher collects all score of students in excel',
 ' The scores are organised in the following categories: Story-telling, Story-writing, Participation, Presentation, Quiz, Listening']

In [ ]:
def lemmatization(texts,allowed_postages=["NOUN","ADJ","VERB","ADV"]):
  nlp=spacy.load("en_core_web_sm",disable=["senter","ner","parser"])
  texts_out=[]
  for text in texts:
    doc=nlp(text)
    new_text=[]
    for token in doc:
      if token.pos_ in allowed_postages:
        new_text.append(token.lemma_)
    final=" ".join(new_text)
    texts_out.append(final)
  return (texts_out)

lemmatized_texts=lemmatization(data)
lemmatized_texts

['first step get datum',
 'teacher collect score student',
 'score organise follow category story telling story writing listen']

In [ ]:
def gen_words(texts):
  final=[]
  for text in texts:
    new=gensim.utils.simple_preprocess(text,deacc=True)
    final.append(new)
  return(final)

data_words=gen_words(lemmatized_texts)
print(data_words)

[['first', 'step', 'get', 'datum'], ['teacher', 'collect', 'score', 'student'], ['score', 'organise', 'follow', 'category', 'story', 'telling', 'story', 'writing', 'listen']]


In [ ]:
id2word=corpora.Dictionary(data_words)
corpus=[]
for text in data_words:
  new = id2word.doc2bow(text)
  corpus.append(new)

print(corpus)

word = id2word
print(word)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1)], [(5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1)]]
Dictionary(15 unique tokens: ['datum', 'first', 'get', 'step', 'collect']...)


In [ ]:
lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=30,random_state=100,update_every=1,
                                           chunksize=100,passes=10,alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,corpus,id2word,mds="mmds",R=30)
vis

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
26     0.045101 -0.163405       1        1  49.441507
16     0.200548  0.151737       2        1  20.462265
11    -0.233519  0.087891       3        1  20.462249
0     -0.000449 -0.002823       4        1   0.356814
28    -0.000449 -0.002823       5        1   0.356814
27    -0.000449 -0.002823       6        1   0.356814
25    -0.000449 -0.002823       7        1   0.356814
24    -0.000449 -0.002823       8        1   0.356814
23    -0.000449 -0.002823       9        1   0.356814
22    -0.000449 -0.002823      10        1   0.356814
21    -0.000449 -0.002823      11        1   0.356814
20    -0.000449 -0.002823      12        1   0.356814
19    -0.000449 -0.002823      13        1   0.356814
18    -0.000449 -0.002823      14        1   0.356814
17    -0.000449 -0.002823      15        1   0.356814
15    -0.000449 -0.002823      16        1   0.356814
1     -0.000449 -0.002823      17        1   0.356814
14    -0.000449 -0.002823      18        1   0.356814
13    -0.000449 -0.002823      19        1   0.356814
12    -0.000449 -0.002823      20        1   0.356814
10    -0.000449 -0.002823      21        1   0.356814
9     -0.000449 -0.002823      22        1   0.356814
8     -0.000449 -0.002823      23        1   0.356814
7     -0.000449 -0.002823      24        1   0.356814
6     -0.000449 -0.002823      25        1   0.356814
5     -0.000449 -0.002823      26        1   0.356814
4     -0.000449 -0.002823      27        1   0.356814
3     -0.000449 -0.002823      28        1   0.356814
2     -0.000449 -0.002823      29        1   0.356814
29    -0.000449 -0.002823      30        1   0.356814, topic_info=        Term      Freq     Total Category  logprob  loglift
12     story  1.000000  1.000000  Default  15.0000  15.0000
5      score  1.000000  1.000000  Default  14.0000  14.0000
8   category  1.000000  1.000000  Default  13.0000  13.0000
9     follow  1.000000  1.000000  Default  12.0000  12.0000
10    listen  1.000000  1.000000  Default  11.0000  11.0000
..       ...       ...       ...      ...      ...      ...
11  organise  0.004044  1.074954  Topic30  -2.7081   0.0529
13   telling  0.004044  1.074954  Topic30  -2.7081   0.0529
14   writing  0.004044  1.074954  Topic30  -2.7081   0.0529
5      score  0.004044  1.847972  Topic30  -2.7081  -0.4889
12     story  0.004044  1.959696  Topic30  -2.7081  -0.5476

[465 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
8         1  0.930273  category
4         3  1.038174   collect
0         2  1.038174     datum
1         2  1.038174     first
9         1  0.930273    follow
2         2  1.038174       get
10        1  0.930273    listen
11        1  0.930273  organise
5         1  0.541134     score
5         3  0.541134     score
3         2  1.038174      step
12        1  1.020566     story
6         3  1.038174   student
7         3  1.038174   teacher
13        1  0.930273   telling
14        1  0.930273   writing, R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[27, 17, 12, 1, 29, 28, 26, 25, 24, 23, 22, 21, 20, 19, 18, 16, 2, 15, 14, 13, 11, 10, 9, 8, 7, 6, 5, 4, 3, 30])